In [14]:
import yaml
import json

def read_yaml_and_emit_dict(file_path):
    with open(file_path, 'r') as stream:
        yaml_documents = yaml.safe_load_all(stream)
        yaml_documents = list(yaml_documents)
        yaml_data = yaml_documents[0]
        yaml_data["customProperties"] = {} if yaml_data["customProperties"] == None else destructureCustomProperty(yaml_data["customProperties"])
        for doc in yaml_documents[1:]:
            for k, v in doc.items():
                if k not in ["id", "name", "customProperties", "artifactType", "registeredModelId"]:
                    yaml_data[k] = v
                if k == "customProperties":
                    yaml_data[k].update(destructureCustomProperty(v))
        return yaml_data
    
def destructureCustomProperty(input: dict)-> dict:
    result = {}
    for k, v in input.items():
        keys_ending_with_value = [key for key in v.keys() if key.endswith("_value")]
        assert len(keys_ending_with_value) == 1
        result[k] = v[keys_ending_with_value[0]]
    return result

def prefix_keys_and_transform_values(dictionary):
    transformed_dict = {}
    for key, value in dictionary.items():
        if isinstance(value, str):
            transformed_dict[key] = value # a string value as-is
        else:
            transformed_dict[f"{key}+json"] = json.dumps(value) # post-fix "+json" for OCI annotation which is a str representing a json
    return transformed_dict


file_path = "example-modelregistry-metadata.yaml"
json_data = read_yaml_and_emit_dict(file_path)
json_data = prefix_keys_and_transform_values(json_data)
json_data = { "$manifest": json_data }

with open("annotations.json", 'w') as f:
    f.write(json.dumps(json_data, indent=2))
